<a href="https://colab.research.google.com/github/poppingary/name-entity-recognition/blob/main/ExtendedCode/flair_harry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import package and load model

In [ ]:
!pip install --upgrade git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-oyhj217z
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-oyhj217z
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 48 kB 4.3 MB/s 
     |████████████████████████████████| 19.7 MB 1.3 MB/s 
     |████████████████████████████████| 788 kB 55.9 MB/s 
     |████████████████████████████████| 1.2 MB 55.2 MB/s 
     |████████████████████████████████| 3.3 MB 31.5 MB/s 
     |████████████████████████████████| 981 kB 48.4 MB/s 
     |████████████████████████████████| 64 kB 2.1 MB/s 
     |████████████████████████████████| 61 kB 428 kB/s 
     |████████████████████████████████| 62 kB 823 kB/s 
     |████████████████████████████████| 3.3 MB 49.4 MB/s 


In [ ]:
from tqdm import tqdm

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load("ner-pooled")

2021-12-12 23:29:25,848 https://nlp.informatik.hu-berlin.de/resources/models/ner-pooled/en-ner-conll03-pooled-v0.5.pt not found in cache, downloading to /tmp/tmpo_pfv_9t


100%|██████████| 1125470069/1125470069 [00:38<00:00, 29001162.27B/s]


2021-12-12 23:30:05,006 copying /tmp/tmpo_pfv_9t to cache at /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:30:09,094 removing temp file /tmp/tmpo_pfv_9t
2021-12-12 23:30:09,253 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt


## load temps and names

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hard_temp_fp = '/content/drive/MyDrive/IntroToMachineLearning/SentenceSample/SentenceTemplate/hard_sentence_templates.txt'
easy_temp_fp = '/content/drive/MyDrive/IntroToMachineLearning/SentenceSample/SentenceTemplate/easy_sentence_templates.txt'
hard_name_fp = '/content/drive/MyDrive/IntroToMachineLearning/SentenceSample/NameSample/rare_names.txt'
easy_name_fp = '/content/drive/MyDrive/IntroToMachineLearning/SentenceSample/NameSample/common_names.txt'

In [ ]:
def read_temp(fp):
  all_string = ""
  with open(fp, 'r', encoding='utf-8-sig') as f:
    for l in f:
      all_string += l
  sents = all_string.split(".")
  return [s.strip()+'.' for s in sents if s != '']

In [ ]:
def read_name(fp):
  names = []
  with open(fp, 'r', encoding='utf-8-sig') as f:
    for l in f:
      names.append(l.strip())
  return names

In [ ]:
easy_temps = read_temp(easy_temp_fp)
hard_temps = read_temp(hard_temp_fp)
easy_names = read_name(easy_name_fp)[:5]
hard_names = read_name(hard_name_fp)[:5] 

## before memory

In [ ]:
def is_correct(entities, name):
    for entity in entities:
      if entity.text == name:
        if entity.tag == 'PER':
          return True
        else:
          return False

In [ ]:
# sent.tokens[0].get_tags_proba_dist('ner')[10].score # 10 is S-PER
def get_per_prob(sent, name):
  for t in sent.tokens:
    if t.text == name:
      s_per = t.get_tags_proba_dist('ner')[10]
      assert s_per.value == 'S-PER'
      return s_per.score

In [ ]:
def before_memory(temp, names, correct, wrong):
  tagger = SequenceTagger.load("ner-pooled")
  for name in names:
    sent_str = temp.replace('*', name)
    sent = Sentence(sent_str)
    tagger.predict(sent, all_tag_prob=True)
    entities = sent.get_spans('ner')
    sPER_score = get_per_prob(sent, name)
    if is_correct(entities, name):
      correct.append((sent, name, sPER_score))
    else:
      wrong.append((sent, name, sPER_score))
  # print("")
  # print("correct: " + str(len(correct)))
  # print("wrong: " + str(len(wrong)))
  # print("precision: " + str(len(correct)/(len(correct) + len(wrong))))
  return correct, wrong 

In [ ]:
def evaluate(correct, wrong):
  def mean_prob(outs):
    total_score = 0
    for sent, name, score in outs:
      total_score += score
    return total_score/len(outs)

  print("correct: " + str(len(correct)))
  print("wrong: " + str(len(wrong)))
  print("precision: " + str(len(correct)/(len(correct) + len(wrong))))
  print("---------------------")
  print("correct prob: " + str(mean_prob(correct)))
  print("wrong prob: " + str(mean_prob(wrong)))
  print("mean prob: " + str(mean_prob(correct+wrong)))

In [ ]:
# easy temp easy name
ee_correct = []
ee_wrong = []
for temp in easy_temps:
  ee_correct, ee_wrong = before_memory(temp, easy_names, ee_correct, ee_wrong)

2021-12-12 23:30:32,029 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:30:41,275 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:30:52,615 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:31:08,690 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:31:18,803 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:31:28,697 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:31:38,997 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:31:48,188 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:31:59,969 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:32:09,562 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:32:18,480 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:32:27,523 loading file /root/

In [ ]:
evaluate(ee_correct, ee_wrong)

correct: 74
wrong: 1
precision: 0.9866666666666667
---------------------
correct prob: 0.9950582232024219
wrong prob: 0.18441706895828247
mean prob: 0.9842496744791667


In [ ]:
# hard temp easy name
# he_correct = []
# he_wrong = []
# for temp in hard_temps:
#   he_correct, he_wrong = before_memory(temp, easy_names, he_correct, he_wrong)
# evaluate(he_correct, he_wrong)

In [ ]:
# easy temp hard name
# eh_correct = []
# eh_wrong = []
# for temp in easy_temps:
#   eh_correct, eh_wrong = before_memory(temp, hard_names, eh_correct, eh_wrong)
# evaluate(eh_correct, eh_wrong)

In [ ]:
# hard temp hard name
hh_correct = []
hh_wrong = []
for temp in hard_temps:
  hh_correct, hh_wrong = before_memory(temp, hard_names, hh_correct, hh_wrong)
evaluate(hh_correct, hh_wrong)

2021-12-12 23:33:08,719 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:33:18,597 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:33:29,187 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:33:39,297 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:33:48,101 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:33:57,074 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:34:07,739 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:34:25,878 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:34:39,549 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:34:51,464 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:35:07,410 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt
2021-12-12 23:35:25,711 loading file /root/

## predict harry potter

In [ ]:
# read harry potter
def read_harry(fp):
  def is_harry(sent):
    if "Harry Potter" in sent:
      sent = sent.replace("Harry Potter", '*')
      return True, sent
    elif "Harry" in sent:
      sent = sent.replace("Harry", '*')
      return True, sent
    elif "Potter" in sent:
      sent = sent.replace("Potter", '*')
      return True, sent
    else:
      return False, sent 

  char_remove = [',', ';', "'s", '@', '&', 
	                '*', '(', ')', '#', '!', 
									'%', '=', '+', '-', '_', 
									':', '"', "'"]

  lines = []
  with open(fp, 'r') as f:
    for l in f:
      l = l.strip()
      for c in char_remove:
        sent = l.replace(c, "")
      is_contain, sent = is_harry(sent)
      if is_contain:
        if len(sent) != 1:
          lines.append(sent)
  return lines  

In [ ]:
harry_fp = '/content/drive/MyDrive/IntroToMachineLearning/SentenceSample/Harry Potter and the Philosopher\'s Stone.txt'

In [ ]:
harry_temps = read_harry(harry_fp)

In [ ]:
len(harry_temps)

1366

## after harry potter

In [ ]:
import random

In [ ]:
random.shuffle(harry_temps)

In [ ]:
def update_memory(sents, tagger):
  for s in tqdm(sents):
    tagger.predict(Sentence(s))
  return tagger

In [ ]:
def after_memory(temps, name, harry_temps):
  def eval(tagger, temps, name):
    correct = []
    wrong = []
    for temp in temps:
      sent_str = temp.replace('*', name)
      sent = Sentence(sent_str)
      tagger.predict(sent, all_tag_prob=True)
      entities = sent.get_spans('ner')
      sPER_score = get_per_prob(sent, name)
      if is_correct(entities, name):
        correct.append((sent, name, sPER_score))
      else:
        wrong.append((sent, name, sPER_score))
    return correct, wrong

  correct_500 = []
  wrong_500 = []
  correct_1000 = []
  wrong_1000 = []

  tagger = SequenceTagger.load("ner-pooled")
  
  ## update memory
  harry_sents = [temp.replace('*', name) for temp in harry_temps]
  tagger = update_memory(harry_sents[:100], tagger)
  correct_500, wrong_500 = eval(tagger, temps, name)
  tagger = update_memory(harry_sents[100:500], tagger)
  correct_1000, wrong_1000 = eval(tagger, temps, name)
  
  return correct_500, wrong_500, correct_1000, wrong_1000

In [ ]:
# easy temp easy name
ee_correct_500_all = []
ee_wrong_500_all = []
ee_correct_1000_all = []
ee_wrong_1000_all = []
for name in easy_names[:1]:
  ee_correct_500, ee_wrong_500, ee_correct_1000, ee_wrong_1000 = after_memory(easy_temps, name, harry_temps)
  ee_correct_500_all += ee_correct_500
  ee_wrong_500_all += ee_wrong_500
  ee_correct_1000_all += ee_correct_1000
  ee_wrong_1000_all += ee_wrong_1000

# eval
print('500')
evaluate(ee_correct_500_all, ee_wrong_500_all)
print('1000')
evaluate(ee_correct_1000_all, ee_wrong_1000_all)

2021-12-12 23:36:39,144 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt


100%|██████████| 400/400 [05:23<00:00,  1.24it/s]


500
correct: 15
wrong: 0
precision: 1.0
---------------------
correct prob: 0.9997792720794678


ZeroDivisionError: ignored

In [ ]:
len(ee_wrong_500_all)

In [ ]:
# hard temp hard name
hh_correct_500_all = []
hh_wrong_500_all = []
hh_correct_1000_all = []
hh_wrong_1000_all = []
for name in hard_names:
  hh_correct_500, hh_wrong_500, hh_correct_1000, hh_wrong_1000 = after_memory(hard_temps, name, harry_temps)
  hh_correct_500_all += hh_correct_500
  hh_wrong_500_all += hh_wrong_500
  hh_correct_1000_all += hh_correct_1000
  hh_wrong_1000_all += hh_wrong_1000

# eval
print('500')
evaluate(hh_correct_500_all, hh_wrong_500_all)
print('1000')
evaluate(hh_correct_1000_all, hh_wrong_1000_all)

2021-12-13 00:10:55,778 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt


100%|██████████| 400/400 [05:19<00:00,  1.25it/s]


2021-12-13 00:18:28,922 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt


100%|██████████| 400/400 [05:36<00:00,  1.19it/s]


2021-12-13 00:26:23,793 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt


...

2021-12-13 00:34:23,946 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt


100%|██████████| 400/400 [05:27<00:00,  1.22it/s]


2021-12-13 00:42:07,217 loading file /root/.flair/models/en-ner-conll03-pooled-v0.5.pt


100%|██████████| 400/400 [05:23<00:00,  1.24it/s]


500
correct: 70
wrong: 5
precision: 0.9333333333333333
---------------------
correct prob: 0.9666580549308232
wrong prob: 0.23236520886421203
mean prob: 0.9177051985263824
1000
correct: 71
wrong: 4
precision: 0.9466666666666667
---------------------
correct prob: 0.9606052435619731
wrong prob: 0.25503035448491573
mean prob: 0.9229745828111966
